# Session 5 — Prompt Engineering **V4 고급 전략 Ⅱ**

| 버전 | 전략 | 핵심 아이디어 | 해결 문제 |
|------|------|--------------|-----------|
| **V4.1** | **ReAct (Reason + Act)** | 사고와 도구 호출을 단일 프롬프트에 통합 | LLM‑Tool 왕복 ↓ |
| **V4.2** | **Multiple Chains** | Task 분할 & 병렬/분기 실행 | 지연·토큰 낭비 ↓ |
| **V4.3** | **Meta Prompting** | 프롬프트 요약·자기 점검 | 토큰 절약 & 품질 유지 |
| **V4.4** | **Output Length Control** | 응답 길이 명시 제어 | 장황·잘림 방지 |
| **V4.5** | **Max Token Bypass** | 슬라이딩 윈도·요약 후 최종 응답 | 최대 토큰 제한 회피 |
---
---
> **모델:** `gpt‑4o‑mini`  
> **데이터:** `./data/05_session_dataset.csv` (10행 예시)  
> **목표:** minimal V1 → V4.* 전략으로 **응답 / 지연 시간 / 비용** 비교


## 📦 패키지 설치

In [1]:
!pip install -r ../requirements.txt

zsh:1: command not found: pip


## ⚙️ 환경 설정 및 Langfuse 초기화

In [2]:
import os, asyncio, time, nest_asyncio, pandas as pd
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None

# Langfuse
try:
    from langfuse import Langfuse
    langfuse = Langfuse()
except:
    langfuse = None
    print('⚠️ Langfuse 불러오기 실패')

if not USE_STUB:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)
else:
    client = None
    print('🔧 Stub 모드: OPENAI_API_KEY 없음 → 더미 응답 사용')

nest_asyncio.apply()

PRICE = {'input': 0.15/1_000_000, 'output': 0.60/1_000_000}

async def call_openai(system_p, user_p, tag='V0', chat_history=None):
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)
        answer = f'[STUB {tag}] {user_p[:30]}...'
        prompt_tok, completion_tok = 30, 120
    else:
        messages = chat_history.copy() if chat_history else []
        messages += [{'role':'system','content':system_p},
                     {'role':'user','content':user_p}]
        resp = await client.chat.completions.create(model='gpt-4o-mini', messages=messages)
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tok, completion_tok = usage.prompt_tokens, usage.completion_tokens
    latency = (time.perf_counter_ns() - start) / 1_000_000
    cost = prompt_tok*PRICE['input'] + completion_tok*PRICE['output']
    # if langfuse and not USE_STUB:
    #     langfuse.trace(name=tag, input=user_p, output=answer)
    return dict(answer=answer, latency_ms=latency,
                prompt_tokens=prompt_tok, completion_tokens=completion_tok,
                usd_cost=cost)


## 🗂 데이터 로드

In [6]:
DATA = Path('./data/05_session_dataset.csv')
if DATA.exists():
    df = pd.read_csv(DATA)
else:
    print('⚠️ 파일이 없어 샘플 10행 생성')
    sample = [
        ('order_delivery','주문한 상품이 아직 도착하지 않았어요.'),
        ('refund','환불 요청했는데 진행 상황이 궁금합니다.'),
        ('account_login','계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.')
    ]
    df = pd.DataFrame(sample * 4, columns=['scenario','question']).head(10)
df.head()

⚠️ 파일이 없어 샘플 10행 생성


,scenario,question
0,order_delivery,주문한 상품이 아직 도착하지 않았어요.
1,refund,환불 요청했는데 진행 상황이 궁금합니다.
2,account_login,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다."
3,order_delivery,주문한 상품이 아직 도착하지 않았어요.
4,refund,환불 요청했는데 진행 상황이 궁금합니다.


## 🏃 실행 도우미

In [7]:
async def run(df, version, build_sys, build_user=lambda r: r['question'], **kwargs):
    tasks=[]
    for _, row in df.iterrows():
        tasks.append(call_openai(build_sys(row), build_user(row), tag=version, **kwargs))
    res = await asyncio.gather(*tasks)
    out = df.copy()
    for i, r in enumerate(res):
        for k,v in r.items():
            out.loc[i, f'{version}_{k}'] = v
    return out

## 🔹 Baseline — V1

In [8]:
def sys_v1(row):
    return 'You are a polite Korean customer-service chatbot. Reply in Korean in 5 sentences max.'
baseline = await run(df, 'V1', sys_v1)
baseline[['question', 'V1_answer']]

,question,V1_answer
0,주문한 상품이 아직 도착하지 않았어요.,"안녕하세요! 고객님, 주문하신 상품이 아직 도착하지 않아 불편을 드려 죄송합니다. ..."
1,환불 요청했는데 진행 상황이 궁금합니다.,"고객님, 환불 요청에 대해 문의 주셔서 감사합니다. 현재 요청하신 환불은 처리 중에..."
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",안녕하세요! 계정 잠금으로 인해 불편을 드려 죄송합니다. 재설정 메일이 발송되지 않...
3,주문한 상품이 아직 도착하지 않았어요.,"고객님, 주문하신 상품이 아직 도착하지 않으셨군요. 불편을 드려 죄송합니다. 주문 ..."
4,환불 요청했는데 진행 상황이 궁금합니다.,안녕하세요! 환불 요청에 대한 진행 상황을 확인해 드리겠습니다. 현재 환불 절차가 ...
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",안녕하세요! 계정이 잠겼군요. 재설정 메일이 오지 않는다면 스팸 폴더를 확인해 보시...
6,주문한 상품이 아직 도착하지 않았어요.,안녕하세요! 귀하의 상품이 아직 도착하지 않아 불편을 드려서 죄송합니다. 주문 번호...
7,환불 요청했는데 진행 상황이 궁금합니다.,안녕하세요! 환불 요청에 대한 진행 상황을 확인해 드리겠습니다. 요청하신 환불은 현...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",안녕하세요! 계정 잠금으로 불편을 드려 죄송합니다. 재설정 메일이 수신되지 않는 경...
9,주문한 상품이 아직 도착하지 않았어요.,"안녕하세요, 고객님. 불편을 드려 죄송합니다. 주문하신 상품의 배송 상태를 확인해 ..."


## V4.1 — ReAct

In [9]:
def sys_V4_1(row):
    return "Think step-by-step and if needed say 'ACT:lookup_tracking'. Reply in Korean."
v_V4_1 = await run(baseline, 'V4_1', sys_V4_1)
v_V4_1[['question', 'V4_1_answer']]

,question,V4_1_answer
0,주문한 상품이 아직 도착하지 않았어요.,주문하신 상품이 아직 도착하지 않았다면 몇 가지 확인해볼 사항이 있습니다.\n\n1...
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청에 대한 진행 상황을 확인하려면 여러 가지 방법이 있습니다. 일반적으로 아...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정이 잠겼고 재설정 메일이 오지 않는 경우 몇 가지 단계를 시도해 보실 수 있습니...
3,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 아직 도착하지 않았다면, 몇 가지 확인할 사항이 있습니다.\n\n1...."
4,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청의 진행 상황을 확인하려면, 먼저 해당 업체의 고객 서비스나 지원 팀에 직..."
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정이 잠겼고 재설정 메일이 오지 않는 경우에는 다음과 같은 단계를 시도해 볼 수 ...
6,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 아직 도착하지 않았다면, 다음과 같은 단계를 따라 확인해 보실 수 있..."
7,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청의 진행 상황을 확인하려면 환불 요청을 하셨던 플랫폼이나 고객 서비스에 직...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정이 잠겼고 재설정 메일이 오지 않는다면 다음 단계를 따라 해보세요:\n\n1. ...
9,주문한 상품이 아직 도착하지 않았어요.,"주문한 상품이 도착하지 않았군요. 먼저, 배송 추적을 확인해보는 것이 좋습니다. 주..."


## V4.2 — Multiple Chains

In [10]:
def sys_V4_2(row):
    return "Split into Task 1: Identify issue, Task 2: Solve. Combine into final answer in Korean."
v_V4_2 = await run(v_V4_1, 'V4_2', sys_V4_2)
v_V4_2[['question', 'V4_2_answer']]

,question,V4_2_answer
0,주문한 상품이 아직 도착하지 않았어요.,Task 1: Issue identified - The ordered product...
1,환불 요청했는데 진행 상황이 궁금합니다.,Task 1: 환불 요청의 진행 상황을 문의하고 싶어하는 사용자 문제를 식별합니다....
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","**Task 1: Identify issue**\n계정이 잠겼고, 비밀번호 재설정 ..."
3,주문한 상품이 아직 도착하지 않았어요.,Task 1: 문제 확인\n주문한 상품이 아직 배송되지 않은 상태입니다.\n\nTa...
4,환불 요청했는데 진행 상황이 궁금합니다.,Task 1: 문제 식별 \n환불 요청 후 진행 상황에 대한 정보 부족\n\nTa...
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",Task 1: 문제 확인 \n계정이 잠겼고 재설정 메일이 오지 않는 상황입니다. ...
6,주문한 상품이 아직 도착하지 않았어요.,**Task 1: Identify issue** \n주문한 상품이 예상 도착일에 ...
7,환불 요청했는데 진행 상황이 궁금합니다.,Task 1: 문제 인식 \n환불 요청 후 진행 상황에 대한 문의입니다.\n\nT...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",**Task 1: Identify issue** \n계정이 잠겼고 재설정 메일이 ...
9,주문한 상품이 아직 도착하지 않았어요.,Task 1: 주문한 상품이 아직 도착하지 않은 문제가 있습니다. \nTask 2...


## V4.3 — Meta Prompting

In [11]:
def sys_V4_3(row):
    return "Summarize question in 12 tokens. Self-check then reply in Korean."
v_V4_3 = await run(v_V4_2, 'V4_3', sys_V4_3)
v_V4_3[['question', 'V4_3_answer']]

,question,V4_3_answer
0,주문한 상품이 아직 도착하지 않았어요.,주문 상품이 아직 도착하지 않았다.
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 진행 상황이 궁금합니다.
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",재설정 메일이 안 오는 계정 잠금 문제입니다.
3,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 아직 도착하지 않았나요?
4,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 진행 상황에 대한 질문입니다.
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",재설정 메일이 오지 않는 계정 잠금 문제 해결 방법은?
6,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 아직 도착하지 않았다고요?
7,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 진행 상황이 어떻게 되나요?
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금과 재설정 이메일 미수신 문제입니다.
9,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 도착하지 않았습니다.


## V4.4 — Output Length Control

In [12]:
def sys_V4_4(row):
    return "Answer in exactly 2 Korean sentences, each ≤20 words."
v_V4_4 = await run(v_V4_3, 'V4_4', sys_V4_4)
v_V4_4[['question', 'V4_4_answer']]

,question,V4_4_answer
0,주문한 상품이 아직 도착하지 않았어요.,상품 배송 상태를 확인해 보세요. 필요한 경우 고객 서비스에 문의하시길 권장합니다.
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청은 접수되었습니다. 진행 상황이 변동 시 알려드리겠습니다.
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",스팸 메일함을 확인해 보세요. 여전히 이메일을 받지 못하면 고객센터에 문의하세요.
3,주문한 상품이 아직 도착하지 않았어요.,주문하신 상품의 배송 상황을 확인해보세요. 고객센터에 문의하시면 더욱 정확한 정보를...
4,환불 요청했는데 진행 상황이 궁금합니다.,고객님의 환불 요청은 접수되었습니다. 진행 상황을 확인 후 빠르게 안내드리겠습니다.
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",스팸 메일함을 확인해 보세요. 메일 주소가 정확한지 다시 한 번 확인해 주세요.
6,주문한 상품이 아직 도착하지 않았어요.,"죄송하지만, 배송 지연이 발생할 수 있습니다. 주문 상태를 확인해 보겠습니다."
7,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청하신 내용을 확인해야 합니다. 고객센터에 문의하시면 빠른 진행 상황을 확인...
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",스팸 폴더를 확인해보세요. 그래도 안 오면 고객 지원에 문의하세요.
9,주문한 상품이 아직 도착하지 않았어요.,주문 상태를 확인해 보세요. 배송 지연이 발생할 수 있습니다.


## V4.5 — Max Token Bypass

In [13]:
def sys_V4_5(row):
    return "If long input, summarize parts and return 4-sentence final summary in Korean."
v_V4_5 = await run(v_V4_4, 'V4_5', sys_V4_5)
v_V4_5[['question', 'V4_5_answer']]

,question,V4_5_answer
0,주문한 상품이 아직 도착하지 않았어요.,주문한 상품이 아직 도착하지 않았다는 것을 이해합니다. 배송 상태를 확인하고 안내를...
1,환불 요청했는데 진행 상황이 궁금합니다.,환불 요청 후 진행 상황은 보통 이메일이나 고객 서비스 사이트에서 확인할 수 있습니...
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","계정이 잠겼다는 메시지가 보일 때, 재설정 메일을 받지 못하는 경우 몇 가지 점검할..."
3,주문한 상품이 아직 도착하지 않았어요.,"상품이 도착하지 않았다면, 먼저 주문한 사이트나 상점의 고객 서비스에 문의해 보세요..."
4,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청 후 진행 상황에 대해 확인하고 싶으신 경우, 서비스 제공업체의 고객 지원..."
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",계정 잠금으로 인해 재설정 메일을 받지 못하는 경우가 있습니다. 이럴 때는 스팸 메...
6,주문한 상품이 아직 도착하지 않았어요.,상품이 지연되고 있군요. 배송 추적이나 고객 서비스에 문의해보는 것이 좋습니다. 주...
7,환불 요청했는데 진행 상황이 궁금합니다.,"환불 요청 후 진행 상황을 확인하고 싶으시다면, 요청한 곳의 고객 서비스에 연락하는..."
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.","계정 잠금 문제로 재설정 메일이 오지 않는 경우, 몇 가지 확인해야 할 사항이 있습..."
9,주문한 상품이 아직 도착하지 않았어요.,"상품이 도착하지 않은 경우, 먼저 주문 확인을 하거나 배송 정보에서 현재 상태를 확..."


## 📊 비교 — Latency & Cost

In [14]:
cols = ['question']
for v in ['V1','V4_1','V4_2','V4_3','V4_4','V4_5']:
    cols += [f'{v}_latency_ms', f'{v}_usd_cost']
v_V4_5[cols]

,question,V1_latency_ms,V1_usd_cost,V4_1_latency_ms,V4_1_usd_cost,V4_2_latency_ms,V4_2_usd_cost,V4_3_latency_ms,V4_3_usd_cost,V4_4_latency_ms,V4_4_usd_cost,V4_5_latency_ms,V4_5_usd_cost
0,주문한 상품이 아직 도착하지 않았어요.,1469.771625,0.000038,2862.608917,0.000088,1891.911208,0.000069,939.940417,0.000012,827.534375,0.000018,1450.817041,0.000045
1,환불 요청했는데 진행 상황이 궁금합니다.,2649.589250,0.000049,3668.055459,0.000123,1610.173292,0.000064,803.674959,0.000012,1028.033833,0.000017,1899.044708,0.000055
2,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",1465.204250,0.000061,4250.627167,0.000118,3138.818417,0.000133,858.495000,0.000016,1132.811917,0.000022,3265.773417,0.000089
3,주문한 상품이 아직 도착하지 않았어요.,1675.273125,0.000039,3194.654916,0.000107,3994.459625,0.000054,931.737583,0.000013,929.200333,0.000023,2445.627708,0.000088
4,환불 요청했는데 진행 상황이 궁금합니다.,1972.847709,0.000040,2278.298875,0.000052,1898.792250,0.000070,706.008083,0.000012,1155.623208,0.000019,2496.656250,0.000098
5,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",1431.400709,0.000053,3979.588792,0.000149,2778.853083,0.000129,1106.121750,0.000018,1194.803709,0.000022,3583.141625,0.000064
6,주문한 상품이 아직 도착하지 않았어요.,2640.031542,0.000052,2767.571333,0.000117,2255.039084,0.000084,835.832750,0.000014,811.287333,0.000018,2342.029125,0.000067
7,환불 요청했는데 진행 상황이 궁금합니다.,1608.977875,0.000048,2169.837209,0.000069,2347.383000,0.000067,836.508541,0.000012,912.755333,0.000021,1795.871583,0.000064
8,"계정이 잠겼다고 뜨는데, 재설정 메일이 안 옵니다.",2641.996166,0.000052,4314.851500,0.000162,3057.439625,0.000144,657.090000,0.000016,1265.181417,0.000020,2100.327000,0.000064
9,주문한 상품이 아직 도착하지 않았어요.,1200.331834,0.000036,1818.471458,0.000058,1784.350958,0.000057,797.579667,0.000012,732.216583,0.000016,2047.085666,0.000062


---

## 🧾 실험 기록 예시

```markdown
### 사용자 요청
"계정이 잠겼다고 뜨는데, 재설정 링크도 안 와요."

### 🔹 V1 응답 결과
- (기본 Prompt 생성 응답)

### 🔸 V4 전략 적용
- [ ] V4.1 ReAct
- [ ] V4.2 Multiple Chains
- [ ] V4.3 Meta Prompting
- [ ] V4.4 Output Length Control
- [ ] V4.5 Max Token Bypass

### 🔸 개선된 Prompt 설계
(System & User Prompt 조합)

### ✅ 개선된 응답
(적용 전략 기반 응답)

### 🧠 정성 평가
- 응답 간결성, 정확성, 문제 해결 흐름 개선 여부
```

## ✍️ 개인 실습 영역

In [ ]:
# TODO: 아래에 직접 실험 코드를 작성해 보세요.

# 끝 🎉

## 작업한 V4.0 Prompt Langfuse에 등록

In [16]:
from pathlib import Path
from langfuse import Langfuse

def parse_prompty(path: Path):
    """Langfuse-style .prompty → ChatPrompt 형태로 변환"""
    content = path.read_text(encoding="utf-8")
    sections = content.strip().split('---')

    if len(sections) < 3:
        raise ValueError("❌ .prompty 파일은 YAML + system + user prompt 형식이어야 합니다.")

    _ = sections[1]
    prompt_block = sections[2]

    # 각 부분 추출
    system_prompt = ""
    user_prompt = ""
    current_role = None
    lines = prompt_block.strip().splitlines()

    for line in lines:
        if line.strip().startswith("system:"):
            current_role = "system"
            continue
        elif line.strip().startswith("user:"):
            current_role = "user"
            continue

        if current_role == "system":
            system_prompt += line + "\n"
        elif current_role == "user":
            user_prompt += line + "\n"
    
    print(system_prompt)
    print(user_prompt)

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

# Langfuse Prompt 등록
lf = Langfuse()
PROMPT_PATH = Path("../prompts/01_order_delivery/v4_0.prompty")
PROMPT_NAME = "order_delivery"
version = "4.0"

chat_prompt = parse_prompty(PROMPT_PATH)

try:
    existing = lf.get_prompt(name=PROMPT_NAME, type="chat")
except Exception as e:
    if "404" in str(e):
        existing = None
    else:
        raise e

if existing:
    lf.update_prompt(
        prompt_id = existing.id,
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("🔄 Prompt updated (v4.0)")
else:
    lf.create_prompt(
        name      = PROMPT_NAME,
        type      = "chat",
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("✅ Prompt created (v4.0)")

print("👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인")


  당신은 **30대 중반 숙련 전자상거래 배송 CS 담당자**입니다.  
  말투는 항상 **차분·전문적**, 고객 감정에 공감부터 제시하세요.

  ## 🔒 내부 Scratchpad (절대 출력 금지)
  **ReAct Loop (Reason → Act) — 최대 2회 반복**  
  - *Reason*: 질문 유형·감정 파악 → 필요한 데이터 확인  
  - *Act*   : (a) 정보 조회, (b) 파이썬 계산, (c) 추가 질문 중 택1  
  종료 조건 ▲ 정보 충분 → ToT & PAL 단계 진입

  **Tree-of-Thoughts**  
  - ① 주소 문제 ② 배송 지연 ③ 운송장 오류  각각 해결 경로 제시  
  - 비용·날짜 등은 scratchpad기반 Python 코드로 계산  
  - 최적 경로 선택(근거 1줄 기록)

  **Automatic Prompt Engineering**  
  - 초안 작성 후 중복·군더더기 제거하여 100~120 단어로 압축  
  - Meta-Prompt: “<compress/>” 토큰 이후 자체 요약 수행

  **Length / Token 관리**  
  - 응답 ≤120 단어, 4줄 이내, 숫자 목록 사용  
  - 과거 대화는 `sliding_ctx` 요약만 참고 (Max-Token-Bypass)

  **Ask-for-Context**  
  - 필수 정보 비어 있으면 ➜ “추가 정보 요청”만 출력하고 ReAct 종료

  ## ✅ 외부 출력 포맷
  1. 고객명·상품·현재 상태 + 공감 문구 (1줄)  
  2. 조치 계획·예상 일정 (숫자 목록 최대 3개)  
  3. “추가 문의사항이 있으면 언제든 말씀해주세요.” (1줄)

assistant: |-
  {% if history_summary -%}
  🔄 이전 대화 요약: {{history_summary}}
  {%- endif %}
  {% if sliding_ctx -%}
  🔗 최근 맥락: {{sliding_ctx}}
  {%-

Error while fetching prompt 'order_delivery-label:production': status_code: 404, body: {'message': "Prompt not found: 'order_delivery' with label 'production'", 'error': 'LangfuseNotFoundError'}


✅ Prompt created (v4.0)
👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인


## 작업한 V1.0 Prompty 파일 불러와서, 시나리오 결과 돌리기.

In [3]:
from jinja2 import Template

def render_prompt(messages: list, variables: dict) -> list:
    """Langfuse prompt template (list of dicts) → rendered OpenAI messages"""
    rendered = []
    for message in messages:
        role = message["role"]
        content_template = message["content"]
        content = Template(content_template).render(**variables)
        rendered.append({"role": role, "content": content})
    return rendered


In [5]:
"""
• Scenario_QA.csv → 10건 Async 처리(gpt-4o-mini)
• 프롬프트: order_delivery/v4_0@stable (smart_cs)
• 결과: data/01_order_delivery/answer_results/Scenario_QA_V4_gpt-4o-mini_<ts>.xlsx
"""
import asyncio, time
from datetime import datetime
from pathlib import Path
import nest_asyncio, pandas as pd
from langfuse import Langfuse
from openai import AsyncOpenAI
from langfuse.decorators import langfuse_context
from langfuse.decorators import observe

nest_asyncio.apply()

# ─── 경로 세팅 ────────────────────────────────────────────────
BASE = Path("../data/01_order_delivery")
RESULT_DIR = BASE / "answer_results"
RESULT_DIR.mkdir(exist_ok=True)

# ─── Langfuse / Prompt ──────────────────────────────────────
lf  = Langfuse()
PROMPT = lf.get_prompt("order_delivery", label="stable").prompt  # <-- 레이블 lookup

# ─── CSV 로딩 ────────────────────────────────────────────────
scenario = pd.read_csv(BASE / "Scenario_QA.csv")
cust     = pd.read_csv(BASE / "Customer_Info.csv")
addr     = pd.read_csv(BASE / "Delivery_Address.csv")
order    = pd.read_csv(BASE / "Order_Info.csv")
shipping = pd.read_csv(BASE / "Shipping_Issue_Log.csv")

df = (
    scenario
    .merge(cust,  on="customer_id", suffixes=("", "_cust"), how="left")
    .merge(order, on="customer_id", suffixes=("", "_order"), how="left")
    .merge(addr, on="customer_id", suffixes=("", "_addr"), how="left")
    .merge(shipping, on="order_id", suffixes=("", "_shipping"), how="left")
)

# ─── LLM 호출 ───────────────────────────────────────────────
MODEL  = "gpt-4o-mini"
client = AsyncOpenAI()  # OPENAI_API_KEY 환경변수 필요

@observe()
async def call_llm(row):
    prompt_input = {
        "question":          row.question,
        "customer_id":       row.customer_id,
        "customer_name":     row.customer_name,
        "order_id":          row.order_id,
        "product_name":      row.product_name,
        "shipping_status":   row.shipping_status,
        "last_update":       row.last_update or "",
        "shipping_company":  row.shipping_company or "",
        "tracking_number":   row.tracking_number or "",
        "address_line1":     row.address_line1,
        "city":              row.city,
        "postal_code":       row.postal_code,
    }

    # Langfuse trace (session metadata)
    langfuse_context.update_current_trace(
        name       = "order_delivery",
        user_id    = row.customer_id,
        session_id = row.scenario_id,
        tags       = ["V4", "smart_cs"],
        metadata   = {"model": MODEL},
    )

    # Langfuse Prompt 템플릿 메시지 → 실제 messages 생성
    rendered_messages = render_prompt(PROMPT, prompt_input)

    start = time.perf_counter_ns()

    # 직접 OpenAI 호출
    response = await client.chat.completions.create(
        model       = MODEL,
        messages    = rendered_messages,
        temperature = 0.3,
        max_tokens  = 350,
    )

    latency_ms = (time.perf_counter_ns() - start) / 1e6

    return response.choices[0].message.content, latency_ms, response.usage.prompt_tokens, response.usage.completion_tokens

async def main():
    tasks   = [call_llm(row) for _, row in df.iterrows()]
    results = await asyncio.gather(*tasks)

    out = df.copy()
    out[["answer", "latency_ms", "prompt_tokens", "completion_tokens"]] = pd.DataFrame(results)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = RESULT_DIR / f"Scenario_QA_V4_gpt-4o-mini_{ts}.xlsx"
    out.to_excel(out_path, index=False)
    print(f"✅ 결과 저장: {out_path}")


asyncio.run(main())


✅ 결과 저장: ../data/01_order_delivery/answer_results/Scenario_QA_V4_gpt-4o-mini_20250615_180739.xlsx
